##### Support Lakebase (v2 autoscaling)

Provision a Lakebase v2 project/branch/endpoint and sync support + credit recommendation data from lakehouse.

In [ ]:
%pip install --upgrade databricks-sdk psycopg2-binary
%restart_python

In [ ]:
import re
import time
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.postgres import Project, ProjectSpec

CATALOG = dbutils.widgets.get("CATALOG")
w = WorkspaceClient()

safe = re.sub(r"[^a-z0-9-]", "-", CATALOG.lower())
safe = re.sub(r"-+", "-", safe).strip("-")
if not safe:
    safe = "support"

project_id = f"{safe}-support-db"
project_name = f"projects/{project_id}"

try:
    project = w.postgres.get_project(project_name)
except Exception:
    op = w.postgres.create_project(
        project_id=project_id,
        project=Project(spec=ProjectSpec(display_name=f"{CATALOG}-support-db")),
    )
    project = op.wait()

branches = list(w.postgres.list_branches(parent=project_name))
default_branch = next((b for b in branches if getattr(getattr(b, "status", None), "default", False)), branches[0])
branch_name = default_branch.name

endpoints = list(w.postgres.list_endpoints(parent=branch_name))
if endpoints:
    endpoint = next((e for e in endpoints if str(getattr(getattr(e, "status", None), "endpoint_type", "")).endswith("READ_WRITE")), endpoints[0])
else:
    # Should normally exist by default; wait and retry a few times before hard fail.
    endpoint = None
    for _ in range(12):
        time.sleep(5)
        endpoints = list(w.postgres.list_endpoints(parent=branch_name))
        if endpoints:
            endpoint = endpoints[0]
            break
    if endpoint is None:
        raise RuntimeError("No Lakebase v2 endpoint found for support-db project")

endpoint_name = endpoint.name
endpoint_host = endpoint.status.hosts.host
print(f"Project: {project_name}")
print(f"Branch: {branch_name}")
print(f"Endpoint: {endpoint_name}")
print(f"Host: {endpoint_host}")

spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.support")
spark.sql(f"""
CREATE OR REPLACE TABLE {CATALOG}.support.lakebase_v2_config (
  updated_at TIMESTAMP,
  project_name STRING,
  branch_name STRING,
  endpoint_name STRING,
  endpoint_host STRING,
  database_name STRING
)
""")
spark.sql(
    f"""
    INSERT OVERWRITE {CATALOG}.support.lakebase_v2_config
    SELECT current_timestamp(), '{project_name}', '{branch_name}', '{endpoint_name}', '{endpoint_host}', 'databricks_postgres'
    """
)

import sys
sys.path.append('../utils')
from uc_state import add

add(CATALOG, "databaseinstances", {"name": project_name, "type": "lakebase_v2_project"})
add(CATALOG, "databasecatalogs", {"name": branch_name, "type": "lakebase_v2_branch"})
add(CATALOG, "pipelines", {"pipeline_id": endpoint_name, "type": "lakebase_v2_endpoint"})

In [ ]:
import psycopg2

creds = w.postgres.generate_database_credential(endpoint=endpoint_name)
current_user = w.current_user.me().user_name
password = creds.token

conn = psycopg2.connect(
    host=endpoint_host,
    port=5432,
    dbname="databricks_postgres",
    user=current_user,
    password=password,
    sslmode="require",
)
conn.autocommit = False

with conn.cursor() as cur:
    cur.execute("CREATE SCHEMA IF NOT EXISTS support")

    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS support.operator_actions (
          action_id BIGSERIAL PRIMARY KEY,
          support_request_id TEXT NOT NULL,
          order_id TEXT NOT NULL,
          user_id TEXT,
          action_type TEXT NOT NULL CHECK (action_type IN ('apply_refund','apply_credit','send_reply')),
          amount_usd NUMERIC(10,2),
          payload JSONB,
          status TEXT NOT NULL DEFAULT 'recorded',
          actor TEXT,
          created_at TIMESTAMPTZ NOT NULL DEFAULT NOW()
        )
        """
    )

    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS support.support_replies (
          reply_id BIGSERIAL PRIMARY KEY,
          support_request_id TEXT NOT NULL,
          order_id TEXT NOT NULL,
          user_id TEXT,
          message_text TEXT NOT NULL,
          sent_by TEXT,
          created_at TIMESTAMPTZ NOT NULL DEFAULT NOW()
        )
        """
    )

    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS support.request_status (
          support_request_id TEXT PRIMARY KEY,
          status TEXT NOT NULL DEFAULT 'pending',
          assigned_to TEXT,
          updated_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
          last_action TEXT,
          notes TEXT
        )
        """
    )

    # Synced support report tables are managed externally (UC synced table),
    # so this stage only provisions app-owned operational tables.

    conn.commit()

conn.close()
print("Support Lakebase v2 provisioning complete")